# Carbon Capture

## Chemical Reaction ( Part 1 Carbamic Acid )

$$NH_3 + CO_2 \rightarrow NH_{2}COOH$$

first of all we want to model the vibrational ground state enegies of the reactants using VQE

In [1]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(
    atom="" \
    "C   0.00000000   0.00000000   0.00000000; " \
    "O   1.20500000   0.00000000   0.00000000; " \
    "O  -0.74624274   1.12787745   0.00000000; " \
    "H  -0.11545599   1.85274770   0.00000000; " \
    "N  -0.78998866  -1.09373606   0.00000000; " \
    "H  -1.18505755  -0.17595546   0.00000000; " \
    "H   0.20010645  -1.22531534   0.00000000; ",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

es_problem = driver.run()

In [2]:
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

transformer = ActiveSpaceTransformer(
    num_electrons=4,
    num_spatial_orbitals=4
)
es_problem = transformer.transform(es_problem)

In [3]:
from qiskit_nature.second_q.mappers import JordanWignerMapper

mapper = JordanWignerMapper()

In [4]:
from qiskit_algorithms import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()

In [5]:
from qiskit_algorithms import VQE
from qiskit.primitives import StatevectorEstimator
from qiskit_algorithms.optimizers import SLSQP
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD


ansatz = UCCSD(
    es_problem.num_spatial_orbitals,
    es_problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
    ),
)

vqe_solver = VQE(StatevectorEstimator(), ansatz, SLSQP())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

In [6]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver(mapper, vqe_solver)

In [7]:
res = calc.solve(es_problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -369.639637663875
  - computed part:      -4.139557238695
  - ActiveSpaceTransformer extracted energy part: -365.50008042518
~ Nuclear repulsion energy (Hartree): 129.438893116141
> Total ground state energy (Hartree): -240.200744547735
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [-5.59409886  3.43617231  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [-5.526140283808  2.691566640373  0.000000506027]
    - computed part:      [2.165574832337  -2.808015379877  0.000000506027]
    - ActiveSpaceTransformer extracted energy part: [-7.691715116146  5.49958202025  0.0]
  > Dipole moment (a.u.): [-0.067958576192  0.744605669627  -0.000000506027]  Total: 0.74770045561
                 (debye): [-0.172733454903  1.892598654396  -0.000001286192]  Total: 1.900464815003
 
